Notebook which will:
- Obtain alert photometry for named SN and convert to AMPEL LightCurve object.
- Match with catalogs from the DESY match service.
- Carry out an SNCosmo fit using the T2RunSncosmo unit.

In [ ]:
import pandas as pd
import requests, os, time
from requests.auth import HTTPBasicAuth
import numpy as np
import matplotlib.pyplot as plt
#import lcdata
from astropy.table import Table
from IPython.core.display import Image, display

In [ ]:
%load_ext ampel_quick_import
%qi AmpelLogger ZTFIdMapper ZiDataPointShaper T1Document LightCurve T2CatalogMatch T2RunSncosmo
#     T2RunSncosmo T2DocView 

In [ ]:
snname = "ZTF21aclhsxb"
desy_archive_user = ""  # Add username 
desy_archive_pwd = ""   # Add pwd

In [ ]:
def api_to_lightcurve(name, archive_user, archive_pw, shaper=None):
    """
    Retrieve the alert history of a SN and convert to LightCurve
    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/object/{}/photopoints".format(name)
    auth = HTTPBasicAuth(archive_user, archive_pw)
    response = requests.get(endpoint, auth=auth )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    # Convert
    alert = response.json()
    if alert is None:
        print(' ... no alert content')
        return None
    
    pps = [alert['candidate']]
    pps.extend( [prv_cand for prv_cand in alert['prv_candidates'] ] )

    if shaper is None:
        tmplog = AmpelLogger.get_logger()
        shaper = ZiDataPointShaper(logger=tmplog)
    
    stockId = ZTFIdMapper.to_ampel_id(name)
    dps = shaper.process( pps, stockId)
    t1d = T1Document(stock=stockId, link=0)
    return LightCurve.build(t1d, dps)

In [ ]:
lc = api_to_lightcurve(snname, desy_archive_user, desy_archive_pwd)

I. Make a catalog match to get a redshift.

In [ ]:
# Match to the DESY ned copy
catconfig = {
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 30.0,
            'catq_kwargs' : {
                'ra_key' : 'RA',
                'dec_key' : 'Dec'
                },
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        }, }

In [ ]:
# Initalize
logger = AmpelLogger.get_logger()
t2cat = T2CatalogMatch( catalogs = catconfig,
    resource = {"ampel-ztf/catalogmatch":'https://ampel.zeuthen.desy.de/api/catalogmatch/'},
    logger=logger)

In [ ]:
# Get a photopoint from which to get coordinates 
t2cat_out = t2cat.process(lc.photopoints[0])

In [ ]:
t2cat_out

In [ ]:
# Use a match if there was one, otherwise enter manually
# z = float( t2out['NEDz']['z'] )

# Manually add a redshift if no NED match was found
z =  0.0326

II. Run Sncosmo using a fixed redshift from the catalog match. Note that in live processing redshifts are  dynamically provied through chaining to the DigestRedshift unit. 

In [ ]:
sncosmo_config = {
    "sncosmo_model_name" : "salt2",
    "backup_z" : z,
    "plot_dir": "/home/jnordin/tmp/ztfsalt",
}

In [ ]:
t2sncosmo = T2RunSncosmo( **sncosmo_config, logger=logger)

In [ ]:
t2sncosmo.post_init()

In [ ]:
sncosmo_out = t2sncosmo.process(lc, t2_views=[])

In [ ]:
sncosmo_out

In [ ]:
fname = "{}/{}_salt2.png".format(sncosmo_config['plot_dir'],snname) 
print(fname)

In [ ]:
display(Image(filename=fname))